In [1]:
# input all the fasta files into assembler per species. make flags to indicate which databases are contributing to outcome. 

# Strains that have only Contigs

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib import rc
import pandas as pd
import sqlite3

cnx = sqlite3.connect(r'/Users/aaronkarlsberg/Desktop/Djoin/data/refSeqFungiStats.db')
strain_with_contigs_only = pd.read_sql_query("SELECT STRAINTAXID FROM speciesdb where chromosome_count = 0 AND contig_count > 0 AND STRAINTAXID !=0 Group by STRAINTAXID HAVING COUNT(DISTINCT DBNAME) > 1", cnx)

print("strain_with_contigs_only", strain_with_contigs_only.size)


# canu requires approximate genome size as input: for now we take average contig length and multiply by the total number of contigs within a single fasta file. The mazimum between fasta files for a species will serve as the approximate genome length.
def generate_canu_command_file_for_strain_contigs(filepaths, contributing_databases, approx_genome_length, strainTaxid):
    approx_genome_length = str(approx_genome_length)
    files = []
    for file in filepaths:
        files.append(file.strip())
    files = ' -i '.join(files)
    print(files)
    distinct_databases = set(contributing_databases)
    distinct_database_count = str(len(distinct_databases))
    contributing_databases = '_'.join(contributing_databases)
    new_file_name = strainTaxid +"_" + distinct_database_count + "_contributing_databases_" + contributing_databases
    shell_script= open("/Users/aaronkarlsberg/Desktop/Djoin/wtdbg_shell_scripts/contig_only_strain_assembly_wtdbg_shell_command_script.sh","a+")
    make_strain_directory = "mkdir /u/home/a/akarlsbe/scratch/strain_contig_only_assemblies/"+ strainTaxid +" && cd /u/home/a/akarlsbe/scratch/strain_contig_only_assemblies/" + strainTaxid +"\n"
    generate_assembly_command_wtdbg2 = "/u/home/a/akarlsbe/wtdbg2/wtdbg2 -x rs -g " + approx_genome_length + " -i "+ files + " -t 16 -fo " + new_file_name + "\n"
    generate_consensus_command_wtpoa = "/u/home/a/akarlsbe/wtdbg2/wtpoa-cns -t 16 -i " + new_file_name + ".ctg.lay.gz -fo "+ new_file_name +".raw.fa\n"
    shell_script.write(make_strain_directory)
    
    shell_script.write(generate_assembly_command_wtdbg2)
    shell_script.write(generate_consensus_command_wtpoa)
    shell_script.close()
    return

for index, row in strain_with_contigs_only.iterrows():
    strain_tax_id = str(row["STRAINTAXID"])
    strain_filepaths_query = pd.read_sql_query("SELECT FILEPATH, DBNAME, contig_count, avg_length_contig FROM speciesdb where STRAINTAXID = "+ strain_tax_id +" AND chromosome_count = 0 AND contig_count > 0 ", cnx)
    strain_filepaths_list = []
    strain_databases = []
    contig_counts = []
    avg_contig_lengths = []
    for indexL, rowL in strain_filepaths_query.iterrows():
        strain_filepaths_list.append(rowL["FILEPATH"])
        strain_databases.append(rowL["DBNAME"])
        contig_counts.append(rowL["contig_count"])
        avg_contig_lengths.append(rowL["avg_length_contig"])
        genome_lengths = []
        for i in range(0,len(contig_counts)):
            genome_lengths.append(contig_counts[i] * avg_contig_lengths[i])
        approx_genome_length = max(genome_lengths)
    generate_canu_command_file_for_strain_contigs(strain_filepaths_list, strain_databases, approx_genome_length, strain_tax_id)  
    
print(strain_filepaths_list, strain_databases, contig_counts,avg_contig_lengths)

#    generate_canu_command = "/u/home/a/akarlsbe/scratch/canu-1.8/*/bin/canu -d /u/home/a/akarlsbe/scratch/strain_contig_only_assemblies/"+ strainTaxid +"_"+distinct_database_count+"_contributing_databases_"+ contributing_databases +" genomeSize="+ approx_genome_length +" -nanopore-raw " + files +" useGrid=false stopOnLowCoverage=2\n"

strain_with_contigs_only 211
/u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/Puccinia_graminisug99.v1.dna.toplevel.fa -i /u/home/a/akarlsbe/scratch/fungi/1K/Pucgr2_AssemblyScaffolds_Repeatmasked.fasta -i /u/home/a/akarlsbe/scratch/fungi/FUNGIDB/FungiDB-41_PgraminisCRL75-36-700-3_Genome.fasta
/u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/Blumeria_graminis.EF2.dna.toplevel.fa -i /u/home/a/akarlsbe/scratch/fungi/1K/Blugr2_AssemblyScaffolds_Repeatmasked.fasta
/u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/Coccidioides_posadasii_c735_delta_sowgp.JCVI-cpa1-1.0.dna.toplevel.fa -i /u/home/a/akarlsbe/scratch/fungi/FUNGIDB/FungiDB-41_CposadasiiC735deltSOWgp_Genome.fasta
/u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/Candida_albicans_sc5314.Cand_albi_SC5314_V4.dna.toplevel.fa -i /u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/Candida_albicans_sc5314_gca_000784655.Cand_albi_SC5314_V3.dna.toplevel.fa -i /u/home/a/akarlsbe/scratch/fungi/1K/Canalb1_AssemblyScaffolds_Repeatmasked.fasta
/u/home/a/akarlsbe/scratch/fungi/ENSEMBLE

/u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/Allomyces_macrogynus_atcc_38327.A_macrogynus_V3.dna.toplevel.fa -i /u/home/a/akarlsbe/scratch/fungi/1K/Allma1_AssemblyScaffolds_Repeatmasked.fasta -i /u/home/a/akarlsbe/scratch/fungi/FUNGIDB/FungiDB-41_AmacrogynusATCC38327_Genome.fasta
/u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/_candida_tenuis_atcc_10573.Candida_tenuis_v1.0.dna.toplevel.fa -i /u/home/a/akarlsbe/scratch/fungi/1K/Cante1_masked_genomic_scaffolds.fasta
/u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/Agaricus_bisporus_var_burnettii_jb137_s8.Agabi_varbur_1.dna.toplevel.fa -i /u/home/a/akarlsbe/scratch/fungi/1K/Abisporus_varburnettii.v2.masked.fasta
/u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/Aspergillus_carbonarius_item_5010.Aspca3.dna.toplevel.fa -i /u/home/a/akarlsbe/scratch/fungi/1K/Aspca3_masked_scaffolds.fasta -i /u/home/a/akarlsbe/scratch/fungi/FUNGIDB/FungiDB-41_AcarbonariusITEM5010_Genome.fasta
/u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/Spathaspora_passalidarum_nrrl_y_27907.Spathaspora

/u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/Mucor_circinelloides_f_lusitanicus_cbs_277_49.Mucci2.dna.toplevel.fa -i /u/home/a/akarlsbe/scratch/fungi/FUNGIDB/FungiDB-41_McircinelloidesCBS277-49_Genome.fasta
/u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/Arthrobotrys_oligospora_atcc_24927.AOL24927_1.0.dna.toplevel.fa -i /u/home/a/akarlsbe/scratch/fungi/1K/Artol1_AssemblyScaffolds_Repeatmasked.fasta
/u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/Pichia_membranifaciens_nrrl_y_2026.Picme2.dna.toplevel.fa -i /u/home/a/akarlsbe/scratch/fungi/1K/Picme2_AssemblyScaffolds_Repeatmasked.fasta
/u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/Phycomyces_blakesleeanus_nrrl_1555_.Phybl2.dna.toplevel.fa -i /u/home/a/akarlsbe/scratch/fungi/FUNGIDB/FungiDB-41_PblakesleeanusNRRL1555_Genome.fasta
/u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/Piloderma_croceum_f_1598.Piloderma_croceum_F_1598_v1.0.dna.toplevel.fa -i /u/home/a/akarlsbe/scratch/fungi/1K/Pilcr1_AssemblyScaffolds_Repeatmasked.fasta
/u/home/a/akarlsbe/scratch/fungi/ENS

/u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/Aspergillus_glaucus_cbs_516_65.Aspgl1.dna.toplevel.fa -i /u/home/a/akarlsbe/scratch/fungi/1K/Aspgl1_AssemblyScaffolds_Repeatmasked.fasta -i /u/home/a/akarlsbe/scratch/fungi/FUNGIDB/FungiDB-41_AglaucusCBS516.65_Genome.fasta
/u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/Puccinia_striiformis_f_sp_tritici_pst_78.P_striiformis_V1.dna.toplevel.fa -i /u/home/a/akarlsbe/scratch/fungi/1K/Pucst_PST78_1_AssemblyScaffolds_Repeatmasked.fasta
/u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/Coniosporium_apollinis_cbs_100218.Coni_apol_CBS100218_V1.dna.toplevel.fa -i /u/home/a/akarlsbe/scratch/fungi/1K/Conap1_AssemblyScaffolds_Repeatmasked.fasta
/u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/Penicillium_digitatum_pd1.PdigPd1_v1.dna.toplevel.fa -i /u/home/a/akarlsbe/scratch/fungi/1K/Pendig1_AssemblyScaffolds_Repeatmasked.fasta
/u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/Capronia_coronata_cbs_617_96.Capr_coro_CBS_617_96_V1.dna.toplevel.fa -i /u/home/a/akarlsbe/scratch/fungi/1K/Ca

In [23]:
# filepath= '/u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/Puccinia_graminisug99.v1.dna.toplevel.fa'

# contig_count = pd.read_sql_query("SELECT DBNAME, contig_count, avg_length_contig FROM speciesdb where straintaxid = 56615 and contig_count>0 and chromosome_count=0 ", cnx)

# print(contig_count)


     DBNAME  contig_count  avg_length_contig
0  ENSEMBLE          3185              37775
1        1K           392             229357
2   FUNGIDB           392             229896


# Strains that have only Chromosomes

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib import rc
import pandas as pd
import sqlite3

cnx = sqlite3.connect(r'/Users/aaronkarlsberg/Desktop/Djoin/data/refSeqFungiStats.db')
strain_with_chromosomes_only = pd.read_sql_query("SELECT STRAINTAXID FROM speciesdb where chromosome_count = 0 AND contig_count > 0 AND STRAINTAXID !=0 Group by STRAINTAXID HAVING COUNT(DISTINCT DBNAME) > 1", cnx)



OperationalError: unable to open database file

# Strains that have both Chromosomes and Contigs within the same fasta file: "Mixed Sequence Files"

In [21]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib import rc
import pandas as pd
import sqlite3

cnx = sqlite3.connect(r'/Users/aaronkarlsberg/Desktop/Djoin/data/refSeqFungiStats.db')
chrom_and_contig_mixed =pd.read_sql_query("SELECT * FROM speciesdb where chromosome_count > 0 AND contig_count > 0 AND STRAINTAXID !=0 Group by STRAINTAXID HAVING COUNT(DISTINCT DBNAME) > 1", cnx)



# Strains that have only chromosomes and only contigs in seperate databases

In [36]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib import rc
import pandas as pd
import sqlite3

cnx = sqlite3.connect(r'/Users/aaronkarlsberg/Desktop/Djoin/data/refSeqFungiStats.db')


strain_with_contigs_in_onedb_chrom_in_other = []

strain_with_contigs_only = pd.read_sql_query("SELECT * FROM SPECIESDB where chromosome_count > 0 AND contig_count = 0 AND STRAINTAXID !=0 Group by STRAINTAXID HAVING COUNT(DISTINCT DBNAME) > 1", cnx)

strain_with_chromosomes_only = pd.read_sql_query("SELECT * FROM SPECIESDB where chromosome_count = 0 AND contig_count > 0 AND STRAINTAXID !=0 Group by STRAINTAXID HAVING COUNT(DISTINCT DBNAME) > 1", cnx)


# for indexCont, rowCont in strain_with_contigs_only.iterrows():
#     for indexChrom, rowChrom in strain_with_chromosomes_only.iterrows():
#         if rowCont["STRAINTAXID"] == rowChrom["STRAINTAXID"]:
#             strain_with_contigs_in_onedb_chrom_in_other.append(rowCont["STRAINTAXID"])

# # Eliminate dublicates
# strain_with_contigs_in_onedb_chrom_in_other = list(dict.fromkeys(strain_with_contigs_in_onedb_chrom_in_other))

# print(strain_with_contigs_in_onedb_chrom_in_other)

# output of the code that is commented out above. This is to avoid running nested for loop more than once. 
strain_with_contigs_in_onedb_chrom_in_other = [227321, 237561, 1071382, 1230383, 871575, 665079, 284813, 425011]



for straintaxid in strain_with_contigs_in_onedb_chrom_in_other:
    pd.read_sql_query("select * from SPECIESDB where straintaxid = "+ str(straintaxid), cnx)









# Species that have only chromosomes and only contigs in seperate databases

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib import rc
import pandas as pd
import sqlite3

cnx = sqlite3.connect(r'/Users/aaronkarlsberg/Desktop/Djoin/data/refSeqFungiStats.db')

species_with_contigs_in_onedb_chrom_in_other = []

species_with_contigs_only = pd.read_sql_query("SELECT * FROM speciesdb where chromosome_count > 0 AND contig_count = 0 AND SPECIESTAXID !=0 Group by SPECIESTAXID HAVING COUNT(DISTINCT DBNAME) > 1", cnx)

species_with_chromosomes_only = pd.read_sql_query("SELECT * FROM speciesdb where chromosome_count = 0 AND contig_count > 0 AND SPECIESTAXID !=0 Group by SPECIESNTAXID HAVING COUNT(DISTINCT DBNAME) > 1", cnx)


# for indexCont, rowCont in species_with_contigs_only.iterrows():
#     for indexChrom, rowChrom in species_with_chromosomes_only.iterrows():
#         if rowCont["SPECIESTAXID"] == rowChrom["SPECIESTAXID"]:
#             species_with_contigs_in_onedb_chrom_in_other.append(rowCont["SPECIESTAXID"])

# # Eliminate dublicates
# species_with_contigs_in_onedb_chrom_in_other = list(dict.fromkeys(species_with_contigs_in_onedb_chrom_in_other))

# print(species_with_contigs_in_onedb_chrom_in_other)

# output of the code that is commented out above. This is to avoid running nested for loop more than once. 
species_with_contigs_in_onedb_chrom_in_other = [746128, 162425, 40559, 5476, 5478, 37769, 432096, 76777, 1005962, 5076, 4932, 5180, 4950, 27337, 4952, 4956, 6035, 109760, 5346, 5062, 61459, 263815, 101852, 1033840, 563466, 29908, 68825, 40302, 498019, 1664694, 42068, 4754, 398673, 254056, 856822, 1367422, 280754, 80884, 5619, 104669, 1209926, 4909, 5127, 5061, 5480]


for speciestaxid in species_with_contigs_in_onedb_chrom_in_other:
    pd.read_sql_query("select * from speciesdb where speciestaxid = "+ str(speciestaxid), cnx)




DatabaseError: Execution failed on sql 'SELECT * FROM speciesdb where chromosome_count = 0 AND contig_count > 0 AND SPECIESTAXID !=0 Group by SPECIESNTAXID HAVING COUNT(DISTINCT DBNAME) > 1': no such column: SPECIESNTAXID

# Use cases to revisit:

In [8]:
# 72 instances in which the fungi_stats code did not characterize the sequence types contained in the fasta files. This is evident where chromosome and contig counts are zero.

unclassified_sequences = pd.read_sql_query("SELECT * FROM speciesdb where chromosome_count = 0 AND contig_count = 0 ", cnx).size



In [9]:
strain_with_contigs_only1 = pd.read_sql_query("SELECT STRAINTAXID FROM speciesdb where chromosome_count = 0 AND contig_count > 0 AND STRAINTAXID !=0", cnx).size
print(strain_with_contigs_only1)

strain_with_contigs_only = pd.read_sql_query("SELECT STRAINTAXID FROM speciesdb where chromosome_count = 0 AND contig_count > 0 AND STRAINTAXID !=0 Group by STRAINTAXID HAVING COUNT(DISTINCT DBNAME) > 0", cnx).size
print(strain_with_contigs_only)

strain_with_contigs_only2 = pd.read_sql_query("SELECT STRAINTAXID FROM speciesdb where chromosome_count = 0 AND contig_count > 0 AND STRAINTAXID !=0 Group by STRAINTAXID HAVING COUNT(DISTINCT DBNAME) = 1", cnx).size
print(strain_with_contigs_only2)

strain_with_contigs_only4 = pd.read_sql_query("SELECT STRAINTAXID FROM speciesdb where chromosome_count = 0 AND contig_count > 0 AND STRAINTAXID !=0 Group by STRAINTAXID HAVING COUNT(DISTINCT DBNAME) > 1", cnx).size
print(strain_with_contigs_only4)


strain_with_contigs_only3 = pd.read_sql_query("SELECT STRAINTAXID FROM speciesdb where chromosome_count = 0 AND contig_count > 0 AND STRAINTAXID !=0 Group by STRAINTAXID HAVING COUNT(STRAINTAXID) > 1", cnx).size

print(strain_with_contigs_only3)


782
541
330
211
213


In [17]:
strain_with_contigs_only5 = pd.read_sql_query("SELECT STRAINTAXID,DBNAME, filepath FROM speciesdb where STRAINTAXID !=0 Group by STRAINTAXID, DBNAME HAVING COUNT(*) > 2", cnx)


print(strain_with_contigs_only5)

example = pd.read_sql_query("SELECT filepath FROM speciesdb where STRAINTAXID= 237561", cnx)
print(example)

   STRAINTAXID    DBNAME                                           FILEPATH
0       237561  ENSEMBLE  /u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/Cand...
                                            FILEPATH
0  /u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/Cand...
1  /u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/Cand...
2  /u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/Cand...
3  /u/home/a/akarlsbe/scratch/fungi/1K/Canalb1_As...
4  /u/home/a/akarlsbe/scratch/fungi/FUNGIDB/Fungi...
